In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.dataset import *
from fastai.structured import *
from fastai.column_data import *

from math import ceil

from dateutil import parser

In [ ]:
torch.cuda.set_device(0)

In [ ]:
import fastai; fastai.__file__

In [ ]:
PATH = "data/"
sz=224
arch=resnext101_64
batch_size=5

# Exploring Other Options

In [1]:
item_categories = pd.read_csv(f'{PATH}item_categories.csv')
items = pd.read_csv(f'{PATH}items.csv')
shops = pd.read_csv(f'{PATH}shops.csv')
train = pd.read_csv(f'{PATH}sales_train.csv')
test = pd.read_csv(f'{PATH}test.csv')

NameError: name 'pd' is not defined

In [ ]:
from itertools import product
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in train['date_block_num'].unique():
    cur_shops = train.loc[train['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = train.loc[train['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

In [ ]:
train = train[train.item_price<100000]
train = train[train.item_cnt_day<=1000]

In [ ]:
train = train.merge(items)

In [ ]:
train = train.merge(item_categories)

In [ ]:
train.head()

In [ ]:
sales_m = sales_m.merge(train)

In [ ]:
sales_m = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': 'sum','item_price': np.mean}).reset_index()
sales_m = pd.merge(grid,sales_m,on=['date_block_num','shop_id','item_id'],how='left').fillna(0)
# adding the category id too
sales_m = pd.merge(sales_m,items,on=['item_id'],how='left')

for type_id in ['item_id','shop_id','item_category_id']:
    for column_id,aggregator,aggtype in [('item_price',np.mean,'avg'),('item_cnt_day',np.sum,'sum'),('item_cnt_day',np.mean,'avg')]:

        mean_df = train.groupby([type_id,'date_block_num']).aggregate(aggregator).reset_index()[[column_id,type_id,'date_block_num']]
        mean_df.columns = [type_id+'_'+aggtype+'_'+column_id,type_id,'date_block_num']

        sales_m = pd.merge(sales_m,mean_df,on=['date_block_num',type_id],how='left')

In [ ]:
sales_m = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': 'sum','item_price': np.mean}).reset_index()
sales_m = pd.merge(grid,train,on=['date_block_num','shop_id','item_id'],how='left').fillna(0)
# adding the category id too
sales_m = pd.merge(sales_m,items,on=['item_id'],how='left')

for type_id in ['item_id','shop_id','item_category_id']:
    for column_id,aggregator,aggtype in [('item_price',np.mean,'avg'),('item_cnt_day',np.sum,'sum'),('item_cnt_day',np.mean,'avg')]:

        mean_df = train.groupby([type_id,'date_block_num']).aggregate(aggregator).reset_index()[[column_id,type_id,'date_block_num']]
        mean_df.columns = [type_id+'_'+aggtype+'_'+column_id,type_id,'date_block_num']

        sales_m = pd.merge(sales_m,mean_df,on=['date_block_num',type_id],how='left')

In [ ]:
lag_variables  = list(sales_m.columns[7:])+['item_cnt_day']
lags = [1 ,2 ,3 ,4, 5, 12]
for lag in lags:
    sales_new_df = sales_m.copy()
    sales_new_df.date_block_num+=lag
    sales_new_df = sales_new_df[['date_block_num','shop_id','item_id']+lag_variables]
    sales_new_df.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variables]
    sales_means = pd.merge(sales_means, sales_new_df,on=['date_block_num','shop_id','item_id'] ,how='left')

## Exploratory Analysis and Feature Engineering

In [ ]:
!ls {PATH}

In [ ]:
sample_subm = pd.read_csv(f'{PATH}sample_submission.csv')
display(sample_subm.tail())

In [ ]:
item_categories = pd.read_csv(f'{PATH}item_categories.csv')
items = pd.read_csv(f'{PATH}items.csv')
shops = pd.read_csv(f'{PATH}shops.csv')
train = pd.read_csv(f'{PATH}sales_train.csv')
test = pd.read_csv(f'{PATH}test.csv')

In [ ]:
test.head()

In [ ]:
# we'll be dropping this shortly, but it will be readded later 
temp_item_price = train['item_price']

In [ ]:
#Make Monthly
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y')
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train = train.drop(['date','item_price'], axis=1)
train = train.groupby([c for c in train.columns if c not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
train = train.rename(columns={'item_cnt_day':'item_cnt_month'})
#Monthly Mean
shop_item_monthly_mean = train[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cnt_month']].mean()
shop_item_monthly_mean = shop_item_monthly_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})
#Add Mean Feature
train = pd.merge(train, shop_item_monthly_mean, how='left', on=['shop_id','item_id'])
#Last Month (Oct 2015)
shop_item_prev_month = train[train['date_block_num']==33][['shop_id','item_id','item_cnt_month']]
shop_item_prev_month = shop_item_prev_month.rename(columns={'item_cnt_month':'item_cnt_prev_month'})
shop_item_prev_month.head()
#Add Previous Month Feature
train = pd.merge(train, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)
#Items features
train = pd.merge(train, items, how='left', on='item_id')
#Item Category features
train = pd.merge(train, item_categories, how='left', on='item_category_id')
#Shops features
train = pd.merge(train, shops, how='left', on='shop_id')
train.head()

In [ ]:
test = test.drop('ID', axis=1)

In [ ]:
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34
#Add Mean Feature
test = pd.merge(test, shop_item_monthly_mean, how='left', on=['shop_id','item_id']).fillna(0.)
#Add Previous Month Feature
test = pd.merge(test, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)
#Items features
test = pd.merge(test, items, how='left', on='item_id')
#Item Category features
test = pd.merge(test, item_categories, how='left', on='item_category_id')
#Shops features
test = pd.merge(test, shops, how='left', on='shop_id')
test['item_cnt_month'] = 0.
test.head()

In [ ]:
train['item_price'] = temp_item_price

In [ ]:
price = train.reset_index().set_index(
    ['item_id', 'shop_id', 'date_block_num']).sort_index()[['item_price', 'item_cnt_month']]
price.head()

In [ ]:
# Helpers
def closest_date_block(current_day, item_id, shop_id):
    """Find the block_date which is closest to the current_day, given item_id and shop_id. Returns index integer"""
    if (item_id, shop_id) in price.index:
        search_lst = np.array(price.loc[(item_id, shop_id)].index)        
        return search_lst[np.abs(current_day - search_lst).argmin()]
    return -1
                
def closest_price(current_day, item_id, shop_id):
    closest_date = closest_date_block(current_day, item_id, shop_id)
    if closest_date != -1:
        return price.loc[( item_id, shop_id, closest_date )]['item_price']
    return np.nan

counter = 0
from IPython.display import clear_output
def closest_price_lambda(x):
    global counter
    if counter % 5000 == 0:
        # hack to display progress, obviously this isn't a
        # robust way to do things :)
        clear_output()
        print("{}/{}".format(counter, len(test)))
    counter += 1
    result = closest_price(34, x.item_id, x.shop_id)
    if counter == len(test):
        clear_output()
        print("DONE")
    return result

In [ ]:
closest_prices = test.apply(closest_price_lambda, axis=1)

In [ ]:
test['item_price'] = closest_prices

# Applying Categorical Variables

In [ ]:
cat_vars = [
    'date_block_num',
    'item_category_id',
    'item_category_name',
    'item_id',
    'item_name',
    'month',              
    'shop_id',
    'shop_name',
    'year',               
]
contin_vars = [
    'item_cnt_month_mean',
    'item_cnt_prev_month',
    'item_cnt_month',
    'item_price',
]
dep = 'item_cnt_month'

In [ ]:
date_block_num_cat = pd.api.types.CategoricalDtype(categories=[i for i in range(35)], ordered=True)
item_category_id_cat = pd.api.types.CategoricalDtype(categories=np.unique(item_categories['item_category_id']), ordered=True)
item_category_name_cat = pd.api.types.CategoricalDtype(categories=np.unique(item_categories['item_category_name']), ordered=True)
item_id_cat = pd.api.types.CategoricalDtype(categories=np.unique(items['item_id']), ordered=True)
item_name_cat = pd.api.types.CategoricalDtype(categories=np.unique(items['item_name']), ordered=True)
month_cat = pd.api.types.CategoricalDtype(categories=np.unique(train['month']), ordered=True)
shop_id_cat = pd.api.types.CategoricalDtype(categories=np.unique(shops['shop_id']), ordered=True)
shop_name_cat = pd.api.types.CategoricalDtype(categories=np.unique(shops['shop_name']), ordered=True)
year_cat = pd.api.types.CategoricalDtype(categories=np.unique(train['year']), ordered=True)

In [ ]:
train['date_block_num'] = train['date_block_num'].astype(date_block_num_cat)
train['item_id'] = train['item_id'].astype(item_id_cat)
train['item_category_id'] = train['item_category_id'].astype(item_category_id_cat)
train['item_category_name'] = train['item_category_name'].astype(item_category_name_cat)
train['item_name'] = train['item_name'].astype(item_name_cat)
train['shop_id'] = train['shop_id'].astype(shop_id_cat)
train['shop_name'] = train['shop_name'].astype(shop_name_cat)
train['month'] = train['month'].astype(month_cat)
train['year'] = train['year'].astype(year_cat)

test['date_block_num'] = test['date_block_num'].astype(date_block_num_cat)
test['item_id'] = test['item_id'].astype(item_id_cat)
test['item_category_id'] = test['item_category_id'].astype(item_category_id_cat)
test['item_category_name'] = test['item_category_name'].astype(item_category_name_cat)
test['item_name'] = test['item_name'].astype(item_name_cat)
test['shop_id'] = test['shop_id'].astype(shop_id_cat)
test['shop_name'] = test['shop_name'].astype(shop_name_cat)
test['month'] = test['month'].astype(month_cat)
test['year'] = test['year'].astype(year_cat)

for v in contin_vars:
    train[v] = train[v].astype('float32')
    test[v] = test[v].astype('float32')


In [ ]:
assert not train.isnull().any().any()

In [ ]:
assert not test.drop('item_price', axis=1).isnull().any().any() # item price can be nan by design

In [ ]:
train.to_pickle(PATH + 'train.pkl')
test.to_pickle(PATH + 'test.pkl')

In [ ]:
!ls {PATH}

# Training

## _Deep Learning Approach_

In [ ]:
train = pd.read_pickle(PATH + 'train.pkl')
test = pd.read_pickle(PATH + 'test.pkl')

In [ ]:
def rmse(y_pred, targ):
    return math.sqrt(((targ - y_pred)**2).mean())

In [ ]:
val = train[train['date_block_num']==33]

In [ ]:
train = train[train['date_block_num']<33]

In [ ]:
df, y, nas, mapper = proc_df(train, dep, do_scale=True)
val_df, val_y, nas, mapper = proc_df(
    val,
    dep,
    do_scale=True,
    mapper=mapper,
)
test_df, _, nas, mapper = proc_df(
    test,
    dep,
    do_scale=True,
    mapper=mapper,
)

In [ ]:
test_df = test_df.drop('item_price_na', axis=1) # ignore these

In [ ]:
y_train = np.log1p(y.clip(0.,40.))
y_val = np.log1p(val_y.clip(0.,40.))

In [ ]:
md = ColumnarModelData.from_data_frames(
    path=PATH,
    trn_df=df,
    trn_y=y_train,
    val_df=val_df,
    val_y=y_val,
    cat_flds=cat_vars,
    bs=256,
    test_df=test_df,
    is_reg=True,  # is regression
    is_multi=False, 
)

In [ ]:
cat_sz = [(c, len(train[c].cat.categories)+1) for c in cat_vars]

In [ ]:
cat_sz

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
emb_szs

In [ ]:
m = md.get_learner(
    emb_szs=emb_szs,
    n_cont=len(df.columns) - len(cat_vars),
    emb_drop=.05,
    out_sz=1,
    szs=[1000, 500],
    drops=[.001, .01],
    use_bn=True
)

In [ ]:
m.lr_find(1e-7)

In [ ]:
m.sched.plot(1)

In [ ]:
lr = 1e-5
m.fit(lrs=lr, n_cycle=3, metrics=[rmse], cycle_len=1, cycle_mult=2)

In [ ]:
m.fit(1e-5, n_cycle=2, metrics=[rmse])

## Submit It!

In [ ]:
result = m.predict(is_test=True)
predictions = np.expm1(result).clip(0, 20)
display(predictions)

In [ ]:
sample_subm.head()

In [ ]:
final_dataframe = pd.read_csv(f'{PATH}test.csv')

In [ ]:
final_dataframe['item_cnt_month'] = predictions
final_dataframe = final_dataframe[['ID', 'item_cnt_month']]

In [ ]:
final_dataframe.head()

In [ ]:
len(final_dataframe)

In [ ]:
len(np.unique(final_dataframe['ID']))

In [ ]:
SUBM = 'sub/'
os.makedirs(SUBM, exist_ok=True)
final_dataframe.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [ ]:
FileLink(f'{SUBM}subm.gz')